In [ ]:
import numpy as np

from matplotlib import pyplot as plt

import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [ ]:
sim_dir = '../final_results/'
sim_id  = '000001'

params = np.load(sim_dir + sim_id  + '/simulation_parameters.npy', allow_pickle=True)[()]

cmab_data = np.load(sim_dir + sim_id + '/mab_statistics.npy', allow_pickle=True)[()]
olla_data = np.load(sim_dir + sim_id + '/olla_statistics.npy', allow_pickle=True)[()]

In [ ]:
env_name = params['env_name']
nrof_tti = params['nrof_tti']
nrof_links = params['nrof_links']
bler_target = params['olla_bler_target']
ann_layout = params[ 'ann_layout' ]

print('%s, %d TTI, %d links, %0.2f BLER target'%(env_name, nrof_tti, nrof_links, bler_target))
print( 'ANN layout: [%s]'%( ','.join( [ str(a) for a in ann_layout ] ) ) )
print( 'Learning rate: %0.6f'%params['learning_rate'])

In [ ]:
def extract_average_cell_metrics( data, nrof_links, interval = 1 ):
    tti = data['tti']

    avg_tput = []
    bler = []
    for res in data['result']:
        tputs = [res[i][1] for i in range(nrof_links)]
        acks = [res[i][3] for i in range(nrof_links)]

        avg_tput.append( np.mean(tputs) * 1000.0 / (6.0 * 15000.0))
        bler.append(1.0 - np.mean(acks))
    
    tti      = tti[ 0 : len( tti ) : interval ]
    avg_tput = avg_tput[ 0 : len( avg_tput ) : interval ]
    bler     = bler[ 0 : len( bler ) : interval ]
        
    return (tti, avg_tput, bler)

In [ ]:
nrof_total_links = 64*256

interval = 5

cmab_tti, cmab_avg_tput, cmab_bler = extract_average_cell_metrics( cmab_data, nrof_total_links, interval )
olla_tti, olla_avg_tput, olla_bler = extract_average_cell_metrics( olla_data, nrof_total_links, interval )

In [ ]:
matplotlib.rcParams.update({'font.size': 40, 'lines.markersize': 15})
cmap = plt.get_cmap("tab10")

plt.figure(figsize=[15, 8])

ax1 = plt.subplot(111)
ax1.grid(True)

lw = 4
ax1.plot(olla_tti, olla_avg_tput, color=cmap(0), marker='o', linewidth=lw)
ax1.plot(cmab_tti, cmab_avg_tput, color=cmap(1), marker='s', linewidth=lw)

ax1.set_ylabel('Average Tput [bps / Hz]')

ax1.set_ylim([1.5, 3])
ax1.set_xlim([0, 100])

ax1.set_xlabel('Transmission Time Interval (TTI)')

plt.legend(['OLLA', 'CMAB'], loc = 'lower right', fontsize=30)